In [ ]:
from RefrigeratedShippingContainer import *

We will introduce a third class to our class heirarchy to create a propery fo which it makes sense to override the setter.

In [ ]:
class HeatedRefrigeratedShippingContainer(RefrigeratedShippingContainer):

    MIN_CELSIUS = -20.0

    @celsius.setter
    def celsius(self, value):
        if not (HeatedRefrigeratedShippingContainer.MIN_CELSIUS 
                <= value
                <= RefrigeratedShippingContainer.MAX_CELSIUS):
            raise ValueError("Temperature out of range")
        self._celsius = value

Unfortunately the celsius object from which which we retrieve the setter decorator is not visible in the scope of the derived class.  We can solve this by fully qualifying thename of the celsius object with the base class name:

In [ ]:
class HeatedRefrigeratedShippingContainer(RefrigeratedShippingContainer):

    MIN_CELSIUS = -20.0

    @RefrigeratedShippingContainer.celsius.setter
    def celsius(self, value):
        if not (HeatedRefrigeratedShippingContainer.MIN_CELSIUS 
                <= value
                <= RefrigeratedShippingContainer.MAX_CELSIUS):
            raise ValueError("Temperature out of range")
        self._celsius = value

In [ ]:
h1 = HeatedRefrigeratedShippingContainer.create_empty('YML', length_ft=40, celsius=-18.0)

In [ ]:
h1

An attempt to set a temperature belwo the minimum via the overriden property causes the ValueError to be raised:

In [ ]:
h1.celsius = -21.0

Similarly attempting to construct an instance with on out-of-range temperature fails as well even though we haven't even defined an __init__() method for the new class.

In [ ]:
h2 = HeatedRefrigeratedShippingContainer.create_empty('YML', length_ft=40, celsius=-25.0)

Recall that the initializer assigns to the underlying _celsius atribute through the celsius property, so our overriden property validator is invoked during construction too, due to polymorphic dispatch

We could ry to eliminate the duplication by delegating the test to the superclass, via super() like this:

In [ ]:
class HeatedRefrigeratedShippingContainer(RefrigeratedShippingContainer):

    MIN_CELSIUS = -20.0

    @RefrigeratedShippingContainer.celsius.setter
    def celsius(self, value):
        if value < HeatedRefrigeratedShippingContainer.MIN_CELSIUS:
            raise ValueError("Temperature too cold!")
        super().celsius = value

In [ ]:
h3 = HeatedRefrigeratedShippingContainer.create_empty('ESC', length_ft=40, celsius=5.0)

For now this is solvable by retrieving the base class property setter function, fset(), from the base class property and calling it directly, remembering to explicitly pass self:

In [ ]:
class HeatedRefrigeratedShippingContainer(RefrigeratedShippingContainer):

    MIN_CELSIUS = -20.0

    @RefrigeratedShippingContainer.celsius.setter
    def celsius(self, value):
        if value < HeatedRefrigeratedShippingContainer.MIN_CELSIUS:
            raise ValueError("Temperature too cold!")
        RefrigeratedShippingContainer.celsius.fset(self, value)

A bonus of implementing this way is that the error messages are slightly more informative, indicating wheter the requested termperature is "too hot" or "too cold", rather than just "out of range"

In [ ]:
h4 = HeatedRefrigeratedShippingContainer.create_empty('ESC', length_ft=40, celsius=-18.0)

In [ ]:
h4.celsius = 10.0

In [ ]:
h4.celsius = -26.0

None of the other code in the class which needs to respect the constraints needs to be modified.  For example, the fahrenheit setter, although not itself overriden, now respects the lower temperature limit:

In [ ]:
h4.fahrenheit = -14.0